<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Split Monopole `GiRaFFEfood` Initial Data for `GiRaFFE`

## Author: Patrick Nelson

### NRPy+ Source Code for this module: [GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_Split_Monopole.py](../../edit/in_progress/GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_Split_Monopole.py)

**Notebook Status:** <font color='green'><b> In-Progress </b></font>

**Validation Notes:** This tutorial notebook has been confirmed to be self-consistent with its corresponding NRPy+ module, as documented [below](#code_validation1). The initial data has validated against the original `GiRaFFE`, as documented [here](Tutorial-Start_to_Finish_UnitTest-GiRaFFEfood_NRPy.ipynb).

## Introduction: 
We need to "feed" our giraffe with initial data to evolve. There are several different choices of initial data we can use here; here, we will only be implementing the "Split Monopole" initial data, given by Table 3 in [the original paper](https://arxiv.org/pdf/1704.00599.pdf). This solution is based on the Blandford-Znajek force-free monopole; it is an approximation for the case of small spin with the solution inverted in the lower hemisphere. The vector potential is
\begin{align}
A_r &= -\frac{aC}{8}\cos\theta \left( 1 + \frac{4M}{r} \right), \\
A_\theta &= 0, \\
A_\phi &= M^2 C [1-\cos \theta + a^2 f(r) \cos \theta \sin^2 \theta],
\end{align}
and the electric field is
\begin{align}
E_r &= -\frac{C a^3}{8\alpha M^3} f'(r) \cos \theta \sin^2 \theta \\
E_\theta &= -\frac{Ca}{8\alpha}[\sin \theta + a^2 f(r) \sin \theta (2 \cos^2 \theta-\sin^2 \theta) ] - \beta^r \sqrt{\gamma} \frac{a C}{8 r^2}\left( 1+\frac{4M}{r}\right) \\
E_\phi &= \frac{\beta^r}{\alpha M} Ca^2 f'(r) \cos \theta \sin^2 \theta,
\end{align}
where
\begin{align}
f(r) =& \ \frac{r^2(2r-3M)}{8M^3} L \left(\frac{2M}{r}\right) \\
&+ \frac{M^2+3Mr-6r^2}{12M^2} \ln \frac{r}{2M} \\
&+ \frac{11}{72} + \frac{M}{3r} + \frac{r}{2M} - \frac{r^2}{M^2}, \\
L(x) =& \ {\rm Li}_2(x) + \frac{1}{2} \ln x \ln (1-x).
\end{align}
The function ${\rm Li}_2(x)$ is known as the dilogarithm function, defined as 
$$ {\rm Li}_2(x) = -\int_{0}^{1} \frac{\ln(1-tx)}{t} dt = \sum_{k=1}^{\infty} \frac{x^k}{k^2}. $$
Now, to use this initial data scheme, we need to transform the above into the quantities actually tracked by `GiRaFFE` and HydroBase: $A_i$, $B^i$, $\tilde{S}_i$, $v^i$, and $\Phi$. Of these quantities, `GiRaFFEfood` will only set $A_i$, $v^i$, and $\Phi=0$, then call a separate function to calculate  $\tilde{S}_i$; `GiRaFFE` itself will call a function to set $B^i$ before the time-evolution begins. This can be done with eqs. 16 and 18, here given in that same order:
\begin{align}
v_{(n)}^i &= \frac{\epsilon^{ijk} E_j B_k}{B^2} \\
B^i &= \frac{[ijk]}{\sqrt{\gamma}} \partial_j A_k \\
\end{align}
In the simulations, $B^i$ will be calculated numerically from $A_i$; however, it will be useful to analytically calculate $B^i$ to use calculating the initial $v^i$.

This module requires the use of the NRPy+ [Shifted Kerr-Schild initial data module](../Tutorial-ADM_Initial_Data-ShiftedKerrSchild.ipynb)

<a id='toc'></a>

# Table of Contents:
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#initializenrpy): Import core NRPy+ modules and set NRPy+ parameters
1. [Step 2](#aux_func): Write helpful auxiliary functions
1. [Step 3](#set_a_i): Set the vector $A_i$
1. [Step 4](#set_vi): Calculate $v^i$ from $B_i$ and $E_i$
1. [Step 5](#code_validation1): Code Validation against `GiRaFFEfood_NRPy.GiRaFFEfood_NRPy` NRPy+ Module
1. [Step 6](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='initializenrpy'></a>

# Step 1: Import core NRPy+ modules and set NRPy+ parameters \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

Here, we will import the NRPy+ core modules after adding NRPy+ to the directory path.

In [1]:
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

# Step 0.a: Import the NRPy+ core modules and set the reference metric to Cartesian
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import sympy as sp               # SymPy: The Python computer algebra package upon which NRPy+ depends
import GiRaFFEfood_NRPy.GiRaFFEfood_NRPy_Common_Functions as gfcf # Some useful functions for GiRaFFE initial data.
# Step 1a: Set commonly used parameters.
thismodule = "GiRaFFEfood_NRPy_Split_Monopole"

# The solution depends on a constant C
C_SM = par.Cparameters("REAL",thismodule,["C_SM"], 1.0) # The wave speed

<a id='aux_func'></a>

# Step 2: Write helpful auxiliary functions \[Back to [top](#toc)\]
$$\label{aux_func}$$

We begin by coding the function $f(r)$ with the inputs $r$ and mass $M$. We avoid calling the function simply `f` out of an abundance of caution, as we do not want to risk overwriting an identically named function elsewhere.
\begin{align}
f(r) =& \ \frac{r^2(2r-3M)}{8M^3} L \left(\frac{2M}{r}\right) \\
&+ \frac{M^2+3Mr-6r^2}{12M^2} \ln \frac{r}{2M} \\
&+ \frac{11}{72} + \frac{M}{3r} + \frac{r}{2M} - \frac{r^2}{M^2}, \\
\end{align}
where $L(x) = {\rm Li}_2 (x) + \frac{1}{2} \ln x \ln (1-x)$ and ${\rm Li}_2 (x)$ is known as the dilogarithm function. We will use the C library `gsl` to handle this special function. In order to do so, we must tell NRPy+ that `nrpyDilog` will be our code word that means "use the `gsl` dilogarithm function, `gsl_sf_dilog`". This is done by simply creating a new sympy function using `Function()` and then adding the name-value pair to the dictionary `custom_functions_for_SymPy_ccode`.

In [2]:
nrpyDilog = sp.Function('nrpyDilog')
from outputC import custom_functions_for_SymPy_ccode
custom_functions_for_SymPy_ccode["nrpyDilog"] = ["gsl_sf_dilog"]

def f_of_r(r,M):
    x = sp.sympify(2)*M/4
    L = nrpyDilog(x) + sp.Rational(1,2)*sp.log(x)*sp.log(sp.sympify(1)-x)
    f = r*r*(sp.sympify(2)*r-sp.sympify(3)*M)*sp.Rational(1,8)/(M**3)*L\
       +(M*M+sp.sympify(3)*M*r-sp.sympify(6)*r*r)*sp.Rational(1,12)/(M*M)*sp.log(r*sp.Rational(1,2)/M)\
       +sp.Rational(11,72) + M*sp.Rational(1,3)/r + r*sp.Rational(1,2)/M - r*r/(M*M)

We will also need the derivative of $f(r)$:
\begin{align}
f'(r) =& \ \frac{2r(2r-3M)+2r^2}{8M^3} L \left(\frac{2M}{r}\right) + \frac{r^2(2r-3M)}{8M^3} L' \left(\frac{2M}{r}\right) \left( -\frac{2M}{r^2} \right) \\
&+ \frac{3M-12r}{12M^2} \ln \frac{r}{2M} + \frac{M^2+3Mr-6r^2}{12M^2} \frac{2M}{r} (2M) \\
&- \frac{M}{3r^2} + \frac{1}{2M} - \frac{2r}{M^2}. \\
\end{align}

Because $$\frac{\partial {\rm Li}_2 (x)}{\partial x} = \frac{{\rm Li}_1 (x)}{x} = \frac{-\ln (1-x)}{x},$$
we know that 
\begin{align}
L'(x) &= \frac{-\ln (1-x)}{x} + \frac{\ln (1-x)}{2x} - \frac{\ln (x)}{2-2x} \\
      &= -\frac{1}{2} \left( \frac{\ln (1-x)}{x} + \frac{\ln(x)}{1-x} \right). 
\end{align}
We simplify this some.
\begin{align}
f'(r) =& \ \frac{6r^2-3M}{8M^3} L \left(\frac{2M}{r}\right) - \frac{2r-3M}{4M^2} L' \left(\frac{2M}{r}\right)\\
&+ \frac{3M-12r}{12M^2} \ln \frac{r}{2M} + \frac{M^2+3Mr-6r^2}{3r} \\
&- \frac{M}{3r^2} + \frac{1}{2M} - \frac{2r}{M^2}. \\
\end{align}


In [3]:
def fp_of_r(x,M):
    x  = sp.sympify(2)*M/4
    L  = nrpyDilog(x) + sp.Rational(1,2)*sp.log(x)*sp.log(sp.sympify(1)-x)
    Lp = -sp.Rational(1,2) * (sp.log(sp.sympify(1)-x)/x + sp.log(x)/(sp.sympify(1)-x))
    f  = (sp.sympify(6)*r*r-sp.sympify(3)*M)*sp.Rational(1,8)/(M**3) + (sp.sympify(2)*r-sp.sympify(3)*M)*sp.Rational(1,4)/(M*M)*Lp\
        +(sp.sympify(3)*M-12*r)*sp.Rational(1,12)/(M*M)*sp.log(r*sp.Rational(1,2)/M) + (M*M+sp.sympify(3)*M*r-sp.sympify(6)*r*r)*sp.Rational(1,3)/r\
        -M*sp.Rational(1,3)/(r*r) + sp.Rational(1,2)/M - 2*r/(M*M)

<a id='set_a_i'></a>

# Step 3: Set the vector $A_i$ \[Back to [top](#toc)\]
$$\label{set_a_i}$$

Now, we will code the components of the vector potential $A_i$ in spherical coordinates and a spherical basis. The outputs from these functions can then be easily converted to other coordinate systems by giving the spherical coordinates as inputs in terms of the desired coordinates (e.g., if we want to use Cartesian coordinates, we pass $r = \sqrt{x^2+y^2+z^2}$ and so on). They can also by transformed into any other basis using the appropriate Jacobian matrix. 

We will code each component as its own function to more easily apply the appropriate staggering.
\begin{align}
A_r &= -\frac{aC}{8}\cos\theta \left( 1 + \frac{4M}{r} \right), \\
\end{align}

In [4]:
def Ar_SM(r,theta,phi, **params):
    C = params["C"]
    M = params["M"]
    # A_r = -aC/8 * cos \theta ( 1 + 4M/r )
    return -a*C*sp.Rational(1,8)*sp.cos(theta)*(sp.sympify(1)+sp.sympify(4)*M/r)

\begin{align}
A_\theta &= 0, \\
\end{align}

In [5]:
def Ath_SM(r,theta,phi, **params):
    # A_\theta = 0
    return sp.sympify(0)

\begin{align}
A_\phi &= M^2 C [1-\cos \theta + a^2 f(r) \cos \theta \sin^2 \theta]
\end{align}

In [6]:
def Aph_SM(r,theta,phi, **params):
    C = params["C"]
    M = params["M"]
    a = params["a"]
    # A_\phi = M^2 C [1-\cos \theta + a^2 f(r) cos \theta sin^2 \theta]
    return M*M*C*(2-sp.cos(theta)+a*a*f_of_r(r,M)*sp.cos(theta)*sp.sin(theta)**2)

<a id='set_vi'></a>

# Step 4: Calculate $v^i$ from $B_i$ and $E_i$ \[Back to [top](#toc)\]
$$\label{set_vi}$$


Next, we will code up the Valencia velocity, which will require us to first code the electric and magnetic fields. The magnetic field is simply $$B^i = \frac{[ijk]}{\sqrt{\gamma}} \partial_j A_k,$$ which gives
\begin{align}
B^r &= \frac{C \alpha M^2}{r^2} + \frac{C \alpha a^2 M^2}{2r^4} \left[ -2\cos \theta + \left(\frac{r}{M}\right)^2 (1+3 \cos 2\theta) f(r) \right], \\                                                                                  
B^\theta &= - \frac{C \alpha a^2}{r^2} \sin \theta \cos \theta f'(r), \\              
B^\phi &= -\frac{C \alpha a M}{8r^2} \left( 1 + \frac{4M}{r}\right) .
\end{align}

The electric field is
\begin{align}
E_r &= -\frac{C a^3}{8\alpha M^3} f'(r) \cos \theta \sin^2 \theta \\
E_\theta &= -\frac{Ca}{8\alpha}[\sin \theta + a^2 f(r) \sin \theta (2 \cos^2 \theta-\sin^2 \theta) ] - \beta^r \sqrt{\gamma} \frac{a C}{8 r^2}\left( 1+\frac{4M}{r}\right) \\
E_\phi &= \frac{\beta^r}{\alpha M} Ca^2 f'(r) \cos \theta \sin^2 \theta,
\end{align}

We can then calculate the the velocity as $$v_{(n)}^i = \frac{\epsilon^{ijk} E_j B_k}{B^2}.$$

In [7]:
def ValenciavU_SM(r,theta,phi, **params):
    C = params["C"]
    M = params["M"]
    a = params["a"]
    alpha = params["alpha"]
    betaU = params["betaU"] # Note that this must use a spherical basis!
    sqrtgammaDET = params["sqrtgammaDET"]

    global BsphU
    BsphU = ixp.zerorank1()
    BsphU[0] = C*alpha*M*M/(r*r) + \
               C*alpha*a*a*M*M*sp.Rational(1,2)/(r**4)*(-sp.sympify(2)*sp.cos(theta) + (r/M)**2*(sp.sympify(1)*sp.sympify(3)*sp.cos(sp.sympify(2)*theta))*f_of_r(r,M))
    BsphU[1] = -C*alpha*a*a/(r*r) * sp.sin(theta)*sp.cos(theta)*fp_of_r(r,M)
    BsphU[2] = -C*alpha*a*M*sp.Rational(1,8)/(r*r)*(sp.sympify(1)+sp.sympify(4)*M/r)

    EsphU = ixp.zerorank1()
    EsphU[0] = -C*a**3/(sp.sympify(8)*alpha*M**3)*fp_of_r(r,M)*sp.cos(theta)*sp.sin(theta)**2
    EsphU[1] = -C*a*sp.Rational(1,8)/alpha*(sp.sin(theta) + a*a*f_of_r(r,M)*sp.sin(theta)*(sp.sympify(2)*sp.cos(theta)**2-sp.sin(theta)**2)) - \
               betaU[0]*sqrtgammaDET*a*C*sp.Rational(1,8)/(r*r)*(sp.sympify(1)+sp.sympify(4)*M/r)
    EsphU[2] = betaU[0]/(alpha*M)*C*a*a*fp_of_r(r,M)*sp.cos(theta)*sp.sin(theta)**2

    EU = gfcf.change_basis_spherical_to_Cartesian(EsphU)
    BU = gfcf.change_basis_spherical_to_Cartesian(BsphU)

    return gfcf.compute_ValenciavU_from_EU_and_BU(EU, BU)

<a id='code_validation1'></a>

# Step 5: Code Validation against `GiRaFFEfood_NRPy.GiRaFFEfood_NRPy` NRPy+ module  \[Back to [top](#toc)\]
$$\label{code_validation1}$$

Here, as a code validation check, we verify agreement in the SymPy expressions for the `GiRaFFE` Exact Wald initial data equations  we intend to use between
1. this tutorial and 
2. the NRPy+ [GiRaFFEfood_NRPy.GiRaFFEfood_NRPy-Split_Monopole](../edit/GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_Split_Monopole.py) module.

In [8]:
from GiRaFFEfood_NRPy.GiRaFFEfood_NRPy import GiRaFFEfood_NRPy_generate_initial_data
import BSSN.ShiftedKerrSchild as sks
sks.ShiftedKerrSchild(True)
import reference_metric as rfm   # NRPy+: Reference metric support
par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
rfm.reference_metric()

# Use the Jacobian matrix to transform the vectors to Cartesian coordinates.
drrefmetric__dx_0UDmatrix = sp.Matrix([[sp.diff(rfm.xxSph[0],rfm.xx[0]), sp.diff(rfm.xxSph[0],rfm.xx[1]), sp.diff(rfm.xxSph[0],rfm.xx[2])],
                                       [sp.diff(rfm.xxSph[1],rfm.xx[0]), sp.diff(rfm.xxSph[1],rfm.xx[1]), sp.diff(rfm.xxSph[1],rfm.xx[2])],
                                       [sp.diff(rfm.xxSph[2],rfm.xx[0]), sp.diff(rfm.xxSph[2],rfm.xx[1]), sp.diff(rfm.xxSph[2],rfm.xx[2])]])
dx__drrefmetric_0UDmatrix = drrefmetric__dx_0UDmatrix.inv()
gammaDD = ixp.zerorank2()
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                gammaDD[i][j] += drrefmetric__dx_0UDmatrix[(k,i)]*drrefmetric__dx_0UDmatrix[(l,j)]*sks.gammaSphDD[k][l].subs(sks.r,rfm.xxSph[0]).subs(sks.th,rfm.xxSph[1])

betaU = ixp.zerorank1()
for i in range(3):
    for j in range(3):
        betaU[i] += dx__drrefmetric_0UDmatrix[(i,j)]*sks.betaSphU[j].subs(sks.r,rfm.xxSph[0]).subs(sks.th,rfm.xxSph[1])
alpha = sks.alphaSph.subs(sks.r,rfm.xxSph[0]).subs(sks.th,rfm.xxSph[1])

GiRaFFEfood_NRPy_generate_initial_data(ID_type = "SplitMonopole", stagger_enable = True,C=C_SM,M=sks.M,a=sks.a,KerrSchild_radial_shift=sks.r0)

r0 + sqrt(xx0**2 + xx1**2 + xx2**2)


UnboundLocalError: local variable 'y' referenced before assignment

<a id='latex_pdf_output'></a>

# Step 6: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GiRaFFEfood_NRPy-Split_Monopole.pdf](Tutorial-GiRaFFEfood_NRPy-Split_Monopole.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [ ]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-GiRaFFEfood_NRPy-Split_Monopole",location_of_template_file=os.path.join(".."))